In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import requests
from pycocotools.coco import COCO

import skimage.io as io

In [2]:
data_dir='./testset'
annotation_file='{}/annotations.json'.format(data_dir)

coco=COCO(annotation_file)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [3]:
catIDs = coco.getCatIds()
cats = coco.loadCats(catIDs)

In [4]:
print(type(cats))
print(cats[1])
print('ID \t: Class Name')
print("-----------------")
for i in cats:
    #print('{}\t: {}'.format(i['id'], i['name']))
    print('{}'.format(i['name']))

<class 'list'>
{'id': 2, 'name': 'back_bumper', 'supercategory': '', 'color': '#dd8271', 'metadata': {}, 'keypoint_colors': []}
ID 	: Class Name
-----------------
_background_
back_bumper
back_glass
back_left_door
back_left_light
back_right_door
back_right_light
front_bumper
front_glass
front_left_door
front_left_light
front_right_door
front_right_light
hood
left_mirror
right_mirror
tailgate
trunk
wheel


In [5]:
print(cats)

[{'id': 1, 'name': '_background_', 'supercategory': '', 'color': '#93a51d', 'metadata': {}, 'keypoint_colors': []}, {'id': 2, 'name': 'back_bumper', 'supercategory': '', 'color': '#dd8271', 'metadata': {}, 'keypoint_colors': []}, {'id': 3, 'name': 'back_glass', 'supercategory': '', 'color': '#dda950', 'metadata': {}, 'keypoint_colors': []}, {'id': 4, 'name': 'back_left_door', 'supercategory': '', 'color': '#ea72a3', 'metadata': {}, 'keypoint_colors': []}, {'id': 5, 'name': 'back_left_light', 'supercategory': '', 'color': '#51fadb', 'metadata': {}, 'keypoint_colors': []}, {'id': 6, 'name': 'back_right_door', 'supercategory': '', 'color': '#ca0f11', 'metadata': {}, 'keypoint_colors': []}, {'id': 7, 'name': 'back_right_light', 'supercategory': '', 'color': '#911ea4', 'metadata': {}, 'keypoint_colors': []}, {'id': 8, 'name': 'front_bumper', 'supercategory': '', 'color': '#2367ef', 'metadata': {}, 'keypoint_colors': []}, {'id': 9, 'name': 'front_glass', 'supercategory': '', 'color': '#64eeb

In [88]:
import json
with open('trainingset/categories.json','w') as outfile:
    color1 =  0
    categorias_gris = []
    for cate in cats:
        ids_temp =cate['id']
        data  = { 
            'id': ids_temp,
            'name': cate['name'],
            'graycolor': color1
        }
        categorias_gris.append(data)
        #json.dump(categorias_gris, outfile)
        color1 += 12
    data  = { 
    'id': ids_temp+1,
    'name': 'car',
    'graycolor': color1+12 }
    categorias_gris.append(data)
    #json.dump(data, outfile)
    data  = { 
    'id': ids_temp+2,
    'name': 'car_damage',
    'graycolor': color1+24 }
    categorias_gris.append(data)
    json.dump(categorias_gris, outfile)

In [5]:
imgIds = coco.getImgIds()
images = coco.loadImgs(imgIds)

Aqui empezamos a implementar metodo que haga la transformación que necesitamos para las clases de una imagen


In [2]:
import json
from PIL import Image, ImageDraw
import os

categorias =  json.loads(open('trainingset/categories.json','r').read())
for id_img in imgIds:
    img = coco.loadImgs(id_img)[0]
    I = io.imread('{}/{}'.format(data_dir,img['path']))/255.0
    #plt.imshow(I)
    #plt.axis('Off')
    alto,ancho, _ = I.shape
    #print(I.shape)
    #print(I.shape[0])
    mascara = Image.new('L',(ancho,alto))
    draw = ImageDraw.Draw(mascara)
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)
    #print(anns[0])
    #print(anns[0]['id'])
    #plt.imshow(I)
    #plt.axis('off')
    coco.showAnns(anns)
    for anotaciones in anns:
        #print(anotaciones)
        class_id = anotaciones['category_id']
        #print(class_id)
        for i in categorias:
            #print(i[0])
            if i["id"] == class_id:
                gray_value = i['graycolor']
                if isinstance(anotaciones['segmentation'], list):
                    for seg in anotaciones['segmentation']:
                        polygon = [ (seg[j], seg[j+1] )  for j in range(0, len(seg), 2)]
                        draw.polygon(polygon, fill = gray_value)    
    #mascara.show()    
    mask_path = os.path.join('trainingset','JPEGMasks')   
    print(mask_path) 

    filename = mask_path+'\mask_'+img['file_name']
    print(filename)
    mascara.save(filename)



NameError: name 'imgIds' is not defined

In [14]:
import json
from PIL import Image, ImageDraw
import os

todo_anotado =  json.loads(open('trainingset/annotations.json','r').read())
categorias =  json.loads(open('trainingset/categories.json','r').read())

#print(todo_anotado["images"])
#print(todo_anotado["categories"])
#print(todo_anotado["annotations"])
cont = 0
previousID = None
pic_abierta=False
for  dato in todo_anotado["annotations"]:
    img_id = dato["image_id"]
    class_id = dato["category_id"]   #clase parte de coche
    if previousID != img_id:
        if pic_abierta == True:
            #hay una foto abierta y con marcas, hay que cerrarla y guardar
            pic_abierta = False
            mask_path = os.path.join('trainingset','JPEGMasks')   
            filename = mask_path+"/"+picname
            print(filename)
            mascara.save(filename)
            

        for names in todo_anotado["images"]:   #buscamos el nombre del archivo poreque ha cambiado
            if names["id"] == img_id:
                pic_abierta = True
                previousID = names["id"]
                picname = names["file_name"]
                picname = picname.replace(".jpg", ".png")
                mascara = Image.new(mode='L',size=(names["height"],names["width"]))
                draw = ImageDraw.Draw(mascara) 
                break
    gray_value = int(class_id -1)
    #for mycolor in categorias:     #buscamos el color en grises 
    #    if mycolor["id"] == class_id:
    #        gray_value = mycolor['graycolor']
    #        break
    if isinstance(dato['segmentation'], list):
        for seg in dato['segmentation']:
            polygon = [ (seg[j], seg[j+1] )  for j in range(0, len(seg), 2)]
            draw.polygon(polygon, fill = gray_value)     


mask_path = os.path.join('trainingset','JPEGMasks')   
filename = mask_path+"/"+picname
print(filename)
mascara.save(filename)

trainingset\JPEGMasks/train1.png
trainingset\JPEGMasks/train10.png
trainingset\JPEGMasks/train100.png
trainingset\JPEGMasks/train101.png
trainingset\JPEGMasks/train102.png
trainingset\JPEGMasks/train103.png
trainingset\JPEGMasks/train104.png
trainingset\JPEGMasks/train105.png
trainingset\JPEGMasks/train106.png
trainingset\JPEGMasks/train107.png
trainingset\JPEGMasks/train108.png
trainingset\JPEGMasks/train109.png
trainingset\JPEGMasks/train11.png
trainingset\JPEGMasks/train110.png
trainingset\JPEGMasks/train111.png
trainingset\JPEGMasks/train112.png
trainingset\JPEGMasks/train113.png
trainingset\JPEGMasks/train114.png
trainingset\JPEGMasks/train115.png
trainingset\JPEGMasks/train116.png
trainingset\JPEGMasks/train117.png
trainingset\JPEGMasks/train118.png
trainingset\JPEGMasks/train119.png
trainingset\JPEGMasks/train12.png
trainingset\JPEGMasks/train120.png
trainingset\JPEGMasks/train121.png
trainingset\JPEGMasks/train122.png
trainingset\JPEGMasks/train123.png
trainingset\JPEGMasks/tra

In [12]:
import json
from PIL import Image, ImageDraw
import os

todo_anotado =  json.loads(open('trainingset/annotations.json','r').read())
categorias =  json.loads(open('trainingset/categories.json','r').read())

#print(todo_anotado["images"])
#print(todo_anotado["categories"])
#print(todo_anotado["annotations"])
cont = 0
previousID = None
pic_abierta=False
for  dato in todo_anotado["annotations"]:
    img_id = dato["image_id"]
    class_id = dato["category_id"]   #clase parte de coche
    if previousID != img_id:
        if pic_abierta == True:
            #hay una foto abierta y con marcas, hay que cerrarla y guardar
            pic_abierta = False
            mask_path = os.path.join('trainingset','JPEGMasks')   
            filename = mask_path+"/"+picname
            #print(filename)
            mascara.save(filename)


        for names in todo_anotado["images"]:   #buscamos el nombre del archivo poreque ha cambiado
            if names["id"] == img_id:
                pic_abierta = True
                previousID = names["id"]
                picname = names["file_name"]
                mascara = Image.new('L',(names["height"],names["width"]))
                draw = ImageDraw.Draw(mascara) 
                break
    for mycolor in categorias:     #buscamos el color en grises 
        if mycolor["id"] == class_id:
            gray_value = mycolor[""]
            if class_id >= 20 or class_id <= 0:
                print("revisar "+filename)
            break
    if isinstance(dato['segmentation'], list):
        for seg in dato['segmentation']:
            polygon = [ (seg[j], seg[j+1] )  for j in range(0, len(seg), 2)]
            draw.polygon(polygon, fill = gray_value)     


mask_path = os.path.join('trainingset','JPEGMasks')   
filename = mask_path+"/"+picname
#print(filename)
mascara.save(filename)
    

In [17]:
256 / 21


12.19047619047619